# Random Number Generation for JIT Compilation

[Chaoming Wang](mailto:chao.brain@qq.com)

Although ``brainpy.math.random`` is designed to be seamlessly compatible with ``numpy.random``, there are still some differences under the context of JIT compilation.

In this section, we are going to talk about how to program a JIT-compatible code with ``brainpy.math.random``.

In [1]:
import brainpy as bp
import brainpy.math as bm
import numpy as np

In [2]:
# bm.set_platform('cpu')

## Using ``bm.random`` outside functions to JIT

Using ``brainpy.math.random`` outside of functions to JIT is the same as using ``numpy.random``.

This usage corresponds to the cases that generating random data for further processing. For example,

In [3]:
bm.random.rand(10)

Array([0.3557365 , 0.9162185 , 0.40441704, 0.7684083 , 0.51694024,
       0.42342532, 0.53246164, 0.95790875, 0.18779063, 0.23019278],            dtype=float32)

In [4]:
np.random.rand(10)

array([0.72133047, 0.82075024, 0.9360612 , 0.8069381 , 0.05206714,
       0.75465615, 0.15721896, 0.46890018, 0.26278736, 0.02719881])

When you are using API functions in ``brainpy.math.random``, actually you are calling functions in a default ``RandomState``. Similarly, ``numpy.random`` also has a default ``RandomState``. Calling a random function in ``numpy.random`` module corresponds to calling the random function in this default NumPy ``RandomState``.

In [5]:
bm.random.DEFAULT

RandomState([2227377921, 2889888023], dtype=uint32)

## Using ``bm.random`` inside a function to JIT

If you are using random sampling in a JIT function, **there are things you need to pay attention to**. Otherwise, the error is likely to raise.

As I have stated above, ``brainpy.math.random`` functions are using the default ``RandomState``. A ``RandomState`` is an instance of brainpy [Variable](./arrays_and_variables.ipynb), denoting that it has values to change after calling any its built-in random function. What's changing is the `key` of a ``RandomState``. For instance,

In [6]:
bm.random.rand(1)
print('Now, the DEFAULT is', bm.random.DEFAULT)

Now, the DEFAULT is RandomState([4041575402, 1508096536], dtype=uint32)


In [7]:
bm.random.rand(1)
print('Now, the DEFAULT is', bm.random.DEFAULT)

Now, the DEFAULT is RandomState([3358813739, 3186405472], dtype=uint32)


Therefore, if you do not specify this DEFAULT ``RandomState`` you are using, repeatedly calling random functions in ``brainpy.math.random`` module will not get what you want, because its `key` cannot be updated. For instance,

In [8]:
@bm.jit
def get_data():
    return bm.random.random(2)

In [9]:
get_data()

Array([0.5392237 , 0.37905133], dtype=float32)

In [10]:
get_data()

Array([0.5392237 , 0.37905133], dtype=float32)

A correct way is explicitly declaring you are using this DEFAULT variable in the JIT transformation.

In [11]:
bm.random.seed()

In [12]:
from functools import partial

@partial(bm.jit, dyn_vars=(bm.random.DEFAULT, ))
def get_data_v2():
    return bm.random.random(2)

In [13]:
get_data_v2()

Array([0.9520792 , 0.31038618], dtype=float32)

In [14]:
get_data_v2()

Array([0.06812155, 0.9231372 ], dtype=float32)

Or, declare the function as a `BrainPyObject`, then use ``jit()``.

In [15]:
@bm.jit
@bm.to_object(dyn_vars=bm.random.DEFAULT)
def get_data_v3():
    return bm.random.random(2)

In [16]:
get_data_v3()

Array([0.8270515, 0.4711758], dtype=float32)

In [17]:
get_data_v3()

Array([0.9246583, 0.881636 ], dtype=float32)

## Using ``RandomState`` for objects to JIT

Another way I recommend is using instances of ``RandomState`` for objects to JIT. For example, you can initialize a ``RandomState`` in the ``__init__()`` function, then using the initialized ``RandomState`` anywhere.

In [18]:
class MyOb(bp.BrainPyObject):
    def __init__(self):
        super().__init__()
        self.rng = bm.random.RandomState(123)

    def __call__(self):
        size = (50, 100)
        u = self.rng.random(size)
        v = self.rng.uniform(size=size)
        z = bm.sqrt(-2 * bm.log(u)) * bm.cos(2 * bm.pi * v)
        return z

In [19]:
ob = bm.jit(MyOb())

In [20]:
ob()

Array([[ 1.3595979 , -1.3462192 ,  0.7149456 , ...,  1.4283268 ,
        -1.1362855 , -0.18378317],
       [-0.26401126, -1.6798397 , -0.8422355 , ...,  1.0795223 ,
         0.41247413, -0.955116  ],
       [ 0.6234829 , -0.44811824, -0.03835859, ..., -2.5203867 ,
        -0.02713326,  1.6490041 ],
       ...,
       [-0.9861029 ,  0.36676335, -0.31499916, ...,  1.526808  ,
        -0.7946268 , -0.86713606],
       [-1.7008592 , -0.05957834, -0.5677447 , ..., -0.04765594,
         0.574145  , -0.11830498],
       [-0.22663854, -1.8517947 , -1.3546717 , ...,  1.2332705 ,
        -0.79247886, -1.9352005 ]], dtype=float32)

Note that any ``Variable`` instance which can be directly accessed by ``self.`` is able to be automatically found by brainpy's JIT transformation functions.
Therefore, in this case, we do not need to pass the ``rng`` into the `dyn_vars` in ``bm.jit()`` function.